In [1]:
# Merge the two dataets
import pandas as pd
import numpy as np

In [4]:
base_dir = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/"

# abundance table
abund = pd.read_csv(f"{base_dir}/data/genus_relative_abundance.csv", sep=",", header=0, dtype={'Unnamed: 0': str})
# rename first column
abund = abund.rename(columns={'Unnamed: 0': 'sample_id'})

growing = pd.read_csv("/home/dermot.kelly/Dermot_primary/Paper_1/data/PACfile_ani_id.csv", sep=",", header=0, low_memory=False)

growing.tail()

,tag,source,ANI_ID,TATTOO,animal_tag,MANAGEMENT_NO,SEX,ANI_ID_DAM,ANI_ID_SIRE,data_origin,...,co2_l_hour_2v3,co2_l_day_2v3,co2_g_hour_2v3,co2_g_day2_2v3,co2_l_hour_1v2,co2_l_day_1v2,co2_g_hour_1v2,co2_g_day2_1v2,flow_rate,Rate_flow
17172,IE046648007714D,CPT,786043.0,.,IE046648007714D,.,F,628731.0,645199.0,N,...,0.396936,571.587913,0.731406,1053.224588,-0.524619,-755.451641,-0.966679,-1392.017264,1.091828,1
17173,IE046648007715F,CPT,785939.0,.,IE046648007715F,.,F,391862.0,758590.0,N,...,0.658713,948.546917,1.208390,1740.080947,-0.651292,-937.860835,-1.194776,-1720.477650,1.011394,1
17174,MGMT96,PED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.197880,284.947200,0.368858,531.155477,-0.296820,-427.420800,-0.553287,-796.733216,0.666667,1
17175,QDA1900841,PED,879766.0,QDA1900841,.,0826176104900841,F,829174.0,766147.0,Y,...,0.620830,893.995916,1.123598,1617.981488,-1.127318,-1623.337454,-2.040254,-2937.966385,0.734286,1
17176,WWB1801202,PED,825644.0,WWB1801202,.,.,F,662978.0,662996.0,Y,...,0.607448,874.725311,1.099378,1583.104951,-0.679437,-978.388732,-1.229665,-1770.718220,0.571464,1


In [5]:
abund['sample_id'] = abund['sample_id'].astype(str)   # make sure it's string

# always treat tag as string first
growing["key7"] = growing["tag"].astype(str).str[-7:].str.zfill(7)
growing["key5"] = growing["tag"].astype(str).str[-5:].str.zfill(5)

growing.head()

,tag,source,ANI_ID,TATTOO,animal_tag,MANAGEMENT_NO,SEX,ANI_ID_DAM,ANI_ID_SIRE,data_origin,...,co2_g_hour_2v3,co2_g_day2_2v3,co2_l_hour_1v2,co2_l_day_1v2,co2_g_hour_1v2,co2_g_day2_1v2,flow_rate,Rate_flow,key7,key5
0,1201,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,-0.913982,-1316.134286,-1.697124,-2443.858587,0.541574,1,0001201,01201
1,1201,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.565561,2254.408230,-1.069258,-1539.731116,-1.981588,-2853.486427,0.838228,1,0001201,01201
2,1201,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0001201,01201
3,1201,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.227683,1767.863908,-0.897100,-1291.824637,-1.605337,-2311.685365,0.758314,1,0001201,01201
4,1201,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.570372,2261.335668,-0.738806,-1063.880597,-1.337517,-1926.024052,0.801641,1,0001201,01201


In [6]:
# Prepare the microbiome data

genus_cols = [c for c in abund.columns if c != "sample_id"]

# ------------------------------------------------------------------
# 3.  PREVALENCE FILTER  (>10 % non-zero)
# ------------------------------------------------------------------
prev = (abund[genus_cols] > 0).mean()
keep = prev[prev >= 0.10].index.tolist()

print(f"Kept {len(keep)} genera out of {len(genus_cols)} (≥10 % prevalence)")


Kept 158 genera out of 309 (≥10 % prevalence)


In [1]:
# ------------------------------------------------------------------
# 4.  LOG-TRANSFORM  (log1p with a small pseudocount)
# ------------------------------------------------------------------
abund_log = abund.copy()
abund_log[keep] = np.log1p(abund_log[keep] + 1e-6)

abund_log.head()


NameError: name 'abund' is not defined

In [8]:
# --- Pass 1: try to match on 7-digit key --------------------------
m7 = abund_log.merge(
        growing,
        left_on="sample_id",
        right_on="key7",
        how="left",
        suffixes=("", "_meta")
     )

# rows where 7-digit **didn’t** match
unmatched = m7[m7["key7"].isna()].copy()

# --- Pass 2: match those leftovers on 5-digit key -----------------
if not unmatched.empty:
    m5 = unmatched.drop(columns=growing.columns, errors="ignore").merge(
            growing,
            left_on="sample_id",
            right_on="key5",
            how="left",
            suffixes=("", "_meta")
        )

    # overwrite the unmatched slice in m7 with the m5 results
    m7.update(m5)

merged = m7


In [9]:
merged.head()

,sample_id,0319-6G20,Absconditabacteriales_(SR1),Acetitomaculum,Agromyces,Alloprevotella,Allorhizobium-Neorhizobium-Pararhizobium-Rhizobium,Alysiella,Anaerobiospirillum,Anaerofustis,...,co2_g_hour_2v3,co2_g_day2_2v3,co2_l_hour_1v2,co2_l_day_1v2,co2_g_hour_1v2,co2_g_day2_1v2,flow_rate,Rate_flow,key7,key5
0,00498,7.696197e-02,0.104361,9.999995e-07,9.999995e-07,7.696197e-02,9.951321e-03,9.951321e-03,9.999995e-07,9.999995e-07,...,0.253844,365.535923,-0.240319,-346.059408,-0.436520,-628.589267,1.199381,1.0,7200498,00498
1,00498,7.696197e-02,0.104361,9.999995e-07,9.999995e-07,7.696197e-02,9.951321e-03,9.951321e-03,9.999995e-07,9.999995e-07,...,0.897590,1292.529439,-0.522191,-751.955582,-0.981434,-1413.264406,1.015529,1.0,8000498,00498
2,00794,6.765958e-02,0.371564,9.999995e-07,2.955977e-02,9.999995e-07,9.999995e-07,9.999995e-07,1.980361e-02,9.999995e-07,...,0.316072,455.143441,-0.240874,-346.858252,-0.439739,-633.223836,0.947088,1.0,7200794,00794
3,00796,2.955977e-02,0.009951,9.951321e-03,9.951321e-03,1.980361e-02,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,...,0.417658,601.427181,-0.263488,-379.422720,-0.477323,-687.345350,0.925926,1.0,7200796,00796
4,00811,9.999995e-07,0.215112,9.999995e-07,2.955977e-02,9.999995e-07,9.999995e-07,9.999995e-07,2.955977e-02,9.999995e-07,...,0.373644,538.047573,-0.312295,-449.704918,-0.570125,-820.980534,0.639387,1.0,7200811,00811


In [10]:
output_path = f"{base_dir}/data/full_growings_microbiome.csv"
merged.to_csv(output_path, index=False)


In [23]:
abund_log.head()

,sample_id,0319-6G20,Absconditabacteriales_(SR1),Acetitomaculum,Agromyces,Alloprevotella,Allorhizobium-Neorhizobium-Pararhizobium-Rhizobium,Alysiella,Anaerobiospirillum,Anaerofustis,...,[Eubacterium]_ruminantium_group,[Eubacterium]_ventriosum_group,[Eubacterium]_xylanophilum_group,[Ruminococcus]_gauvreauii_group,[Ruminococcus]_gnavus_group,hoa5-07d05_gut_group,p-251-o5,probable_genus_10,uncultured,vadinBE97
0,00498,7.696197e-02,0.104361,9.999995e-07,9.999995e-07,7.696197e-02,9.951321e-03,9.951321e-03,9.999995e-07,9.999995e-07,...,0.019804,2.955977e-02,5.826985e-02,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,1.311032,1.043609e-01
1,00794,6.765958e-02,0.371564,9.999995e-07,2.955977e-02,9.999995e-07,9.999995e-07,9.999995e-07,1.980361e-02,9.999995e-07,...,0.058270,8.617861e-02,5.826985e-02,9.999995e-07,9.999995e-07,9.999995e-07,9.951321e-03,9.999995e-07,1.790092,9.951321e-03
2,00796,2.955977e-02,0.009951,9.951321e-03,9.951321e-03,1.980361e-02,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,...,0.198852,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,1.144223,3.922167e-02
3,00811,9.999995e-07,0.215112,9.999995e-07,2.955977e-02,9.999995e-07,9.999995e-07,9.999995e-07,2.955977e-02,9.999995e-07,...,0.285180,5.826985e-02,1.980361e-02,9.999995e-07,1.980361e-02,9.999995e-07,2.955977e-02,3.922167e-02,2.106570,9.999995e-07
4,00812,9.999995e-07,0.029560,1.980361e-02,9.951321e-03,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,9.999995e-07,...,0.019804,7.696197e-02,9.999995e-07,1.980361e-02,9.999995e-07,9.951321e-03,9.999995e-07,9.531109e-02,1.566531,3.922167e-02
